In [4]:
import argparse
from datetime import datetime
import logging
import os

import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions

In [5]:
PROJECT = 'sandbox-michael-menzel'
REGION = 'europe-west4'
BUCKET = 'gs://sandbox-michael-menzel-training-europe-west4/staging'

DATA_FILE = 'gs://sandbox-michael-menzel-data-europe-west4/advertising-regression-data/advertising.csv'

PIPELINE_DISPLAY_NAME = f'dataflow-adhoc-{int(datetime.now().timestamp())}'

In [6]:
table_schema = {
    'fields': [
        {
            "name": "TV",
            "type": "FLOAT",
            "mode": "REQUIRED"
        },
        {
            "name": "Radio",
            "type": "STRING",
            "mode": "REQUIRED"
        },
        {
            "name": "Newspaper",
            "type": "FLOAT",
            "mode": "REQUIRED"
        },
        {
            "name": "Sales",
            "type": "FLOAT",
            "mode": "REQUIRED"
        },
    ]}

def parse_file(element):
    import csv
    for TV, radio, newspaper, sales in csv.reader([element], quotechar='"', delimiter=',', quoting=csv.QUOTE_ALL, skipinitialspace=True):
        return {'TV': TV, 'Radio': radio, 'Newspaper': newspaper, 'Sales': sales}

    

options = PipelineOptions(
    job_name=PIPELINE_DISPLAY_NAME,
    save_main_session=True,
    runner='DataflowRunner',
    project = PROJECT,
    region = REGION,
    temp_location = BUCKET,
    staging_location = BUCKET,
)

with beam.Pipeline(options=options) as pipeline:
    (
        pipeline
        | 'Read CSV from GCS' >> beam.io.ReadFromText(DATA_FILE)
        | 'Parse CSV file' >> beam.Map(parse_file)
        | 'Write To BigQuery' >> beam.io.WriteToBigQuery(
            'example_dataset.advertising_regression',
            schema=table_schema,
            write_disposition=beam.io.BigQueryDisposition.WRITE_TRUNCATE,
            create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED)
    )

/home/jupyter/.local/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery.py:2102: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  self.table_reference.projectId = pcoll.pipeline.options.view_as(
/home/jupyter/.local/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery_file_loads.py:1112: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = p.options.view_as(GoogleCloudOptions).temp_location
